In [1]:
!!pip3 install emoji

['Collecting emoji',
 'deeppavlov 0.0.5 requires tensorflow==1.8.0, which is not installed.',
 'Installing collected packages: emoji',
 'Successfully installed emoji-0.5.0']

In [1]:
import json
import re
from nltk.tokenize import wordpunct_tokenize
import csv
import numpy as np
from emoji import UNICODE_EMOJI
import dateutil.parser
import html
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

DATA_INPUT_FILE = '/data/NER/VectorX/documents_description_objects_100K_random.json'
DATA_ALL_OUTPUT_FILE = '/data/NER/VectorX/dataset.csv'

In [2]:
san_tokenizer = re.compile(r"[\w']+|[^\w ]")

def tokenize(s):
    return san_tokenizer.findall(s)

In [3]:
def has_emoji(s):
    for emoji in UNICODE_EMOJI:
        if emoji in s:
            return True
    return False

_bad_beginnings = ['[id', 'Фото', 'Смотрите', 'Скачать', 'Оригинал взят']
_bad_substrings = ['!!', '...']
def check(t: str) -> bool:
    for bb in _bad_beginnings:           
        if t.startswith(bb):
            return False
    for bs in _bad_substrings:
        if bs in t:
            return False
    
    if has_emoji(t):
        return False
    
    return t.count('(') == t.count(')')

In [6]:
%%time
lines = []
data = []

san_dquoter = re.compile(r"(«|»|“|”|``|'|„|“)")

NumberInt = re.compile(r'NumberInt\((\d+)\)')
ISODate = re.compile(r'ISODate\(("[^"]+")\)')

quoter = re.compile(r'& ?[qQ]uot ?;')
amper = re.compile(r'& ?amp ?; ?')
words_re = re.compile(r'\w+')

photo_re = re.compile(r'[Фф]ото: [\.a-zA-Z0-9\-]+\.[a-zA-Z]+')

counter = 0

descr_set = set()

with open(DATA_INPUT_FILE) as f:
    for line in f:
        line = line.rstrip()        
        m1 = NumberInt.findall(line)
        if m1:
            line = NumberInt.sub(m1[0], line)
        m2 = ISODate.findall(line)
        if m2:
            line = ISODate.sub(m2[0], line)
            
        lines.append(line)
        if line == '}':
#             print('\n'.join(lines))
            j_orig = json.loads('\n'.join(lines))
            pub_date = j_orig['rss_pubDate']
        
            assert len(j_orig) == 2
            j = j_orig['rss']            
            assert len(j) == 1
            j = j['channel']
            assert len(j) == 1
            j = j['item']
            
            raw_description = j['description']
            guid = j['guid']
            objects = j['objects'].get('object', [])

            
            if raw_description and isinstance(raw_description, str):
                # assert isinstance(description, str), f'"{description}" is not str in object {j_orig}'
                description = html.unescape(quoter.sub('&quot;', amper.sub('&', photo_re.sub('', raw_description))))
                description = san_dquoter.sub('"', description)
                
                words_only = tuple(words_re.findall(description.lower()))
                if words_only not in descr_set:
                    descr_set.add(words_only)
                    tokenized = tokenize(description)
                    description_tokenized = ' '.join(tokenized)
                    data.append([guid, description, description_tokenized, len(tokenized), pub_date, objects])            
                    counter += 1

            lines = []
print(f'Read {counter} articles')

Read 0 articles
CPU times: user 601 µs, sys: 302 µs, total: 903 µs
Wall time: 576 µs


In [7]:
j

{'guid': 82387388,
 'description': 'Проблемное снабжению холодной и горячей воды в некоторых домах Винницы проявляется через дырявые трубы. Заявляют в & quot; Винницагортеплоэнерго & quot;. Подсчитывают, только за полгода одном из домов в городе, вытекло около 2000 кубометров воды. Предприятие разработало жесткий алгоритм устранения проблемы.\n& Quot; Провели aнaлиз, и оказалось, что около 20 000 кубических метров было потерь во внутренних тепловых мережaх при домивкaх. Это потому, что ЖЭКи проводили плохо експлуaтaцию. Приклaд - Вaтутинa, 35. Затем мы знaйшлы еще подобные инциденты по другим улицам. Трaсы тaм в жaхливому Стaнь. Нa одном метре трубы по 5-6 резиновых стяжек. Трубы те уже експлуaтувaты неможнa, a они используются. Втрaты воды идут. Только по одному дому около 2000 кубометров вытекло зa полгода. Это понaд 140000 гривен мы потеряли по одному помещению. A подобных домов в еще бaгaто & quot ;, - розповидaе Вiнниця.info Юлиaн Мигaенко, нaчaльник службы безопасности кaдрового 

In [95]:
%%time

cv = CountVectorizer()
texts = [description_tokenized.lower() for guid, description, description_tokenized, token_count, pub_date, objects in data]
counts = cv.fit_transform(texts)    

single_doc_words_count = csr_matrix(single_doc_words).multiply(csr_matrix(counts > 0)).sum(axis=1) 
uniq_words_counts = (counts > 0).sum(axis=1)
single_doc_words_ratio = np.array(single_doc_words_count / np.maximum(1, uniq_words_counts))[:, 0]

CPU times: user 19.1 s, sys: 167 ms, total: 19.3 s
Wall time: 19.3 s


In [96]:
%%time
with open(DATA_ALL_OUTPUT_FILE, 'w') as fw:
    cw = csv.writer(fw)
    cw.writerow(['guid', 'descriptions', 'description_tokenized', 'token_count', 'pub_date', 'objects', 'single_doc_words_ratio'])
    for row, sdwr in zip(data, single_doc_words_ratio):
        cw.writerow(row + [sdwr])       

CPU times: user 7.55 s, sys: 620 ms, total: 8.17 s
Wall time: 10.6 s


In [99]:
%%time

target_date = datetime(2017, 8, 1).replace(tzinfo=pytz.UTC).date()
# target_date = datetime(2017, 8, 31).replace(tzinfo=pytz.UTC).date()
toloka_file = DATA_TOLOKA_FILE_TEMPLATE.format(target_date)

print(f'Writing to file {toloka_file}')

c = 0
with open(toloka_file, 'w') as fw:
    cw = csv.writer(fw, delimiter='\t')
    cw.writerow(['INPUT:guid', 'INPUT:orig', 'INPUT:input'])
    for guid, description, description_tokenized, token_count, pub_date, objects, sdwr in data:        
        row_date = dateutil.parser.parse(pub_date).date()
        if 40 < int(token_count) < 1750 and row_date == target_date and float(sdwr) < 0.15:
            c += 1
            cw.writerow([guid, description, description_tokenized])
print(f'wrote {c} articles')

Writing to file /data/NER/VectorX/toloka_2017-08-01.tsv
wrote 3762 articles
CPU times: user 6.88 s, sys: 113 ms, total: 6.99 s
Wall time: 7.01 s
